In [1]:
import networkx as nx
import torch
import vtk
from vtk import vtkXMLPolyDataReader
#import pymeshlab as pm
import re

import numpy as np
import vtk
from vtk import vtkXMLPolyDataReader, vtkPolyDataReader
from vmtk import vmtkscripts
import torch
import time
#import pymeshlab as pm
#from vtp2obj import vtpToObj
import trimesh
seed = int(time.time())
import os
#import trimesh

In [2]:
import vtk
import numpy as np

def create_vtk_polydata(segments, radii):
    # Create vtkPoints and vtkCellArray to store points and lines
    points = vtk.vtkPoints()
    cellarray = vtk.vtkCellArray()
    radius_array = vtk.vtkFloatArray()
    radius_array.SetName("Radius")


    

    # Dictionary to store point IDs to avoid duplicates
    point_id_map = {}

    # Iterate over each segment and its corresponding radius
    for segment, radius_segment in zip(segments, radii):
        polyLine = vtk.vtkPolyLine()
        for i, (point, radius) in enumerate(zip(segment, radius_segment)):
            # Convert point to tuple for hashing
            point_tuple = tuple(point)

            # Check if the point already exists, else insert it
            if point_tuple not in point_id_map:
                point_id = points.InsertNextPoint(point)
                point_id_map[point_tuple] = point_id
                radius_array.InsertNextValue(radius)  # Assuming radius needs to be halved
            else:
                point_id = point_id_map[point_tuple]

            # Add the point ID to the polyline
            polyLine.GetPointIds().InsertNextId(point_id)

        # Insert the polyline into the cell array
        if polyLine.GetPointIds().GetNumberOfIds() > 1:
            cellarray.InsertNextCell(polyLine)

    # Create the vtkPolyData and add points and lines
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetLines(cellarray)
    polydata.GetPointData().AddArray(radius_array)

    return polydata

def load_segments_and_radii_from_text(segment_filename, radii_filename):
    # Load segments from a text file
    segments = []
    with open(segment_filename, 'r') as seg_file:
        current_segment = []
        for line in seg_file:
            if line.startswith("# New Segment"):
                if current_segment:
                    segments.append(np.array(current_segment))
                    current_segment = []
            else:
                current_segment.append(list(map(float, line.strip().split(','))))
        if current_segment:
            segments.append(np.array(current_segment))
    
    # Load radii from a text file
    radii = []
    with open(radii_filename, 'r') as rad_file:
        current_radii = []
        for line in rad_file:
            if line.startswith("# New Radii"):
                if current_radii:
                    radii.append(np.array(current_radii).flatten())
                    current_radii = []
            else:
                current_radii.extend(list(map(float, line.strip().split(','))))
        if current_radii:
            radii.append(np.array(current_radii).flatten())
    
    return segments, radii


def process_mesh(input_path, output_path, reduction_factor=0.2):
    """
    Processes an OBJ mesh: inverts face orientations, optionally smooths, and simplifies the mesh.

    Parameters:
    - input_path (str): Path to the input OBJ file.
    - output_path (str): Path to save the processed OBJ file.
    - target_faces (int): Target number of faces for simplification. If None, no simplification is applied.

    Returns:
    - None: The processed mesh is saved to `output_path`.
    """
    # Load the mesh
    mesh = trimesh.load(input_path, force='mesh')
    
    if not isinstance(mesh, trimesh.Trimesh):
        raise ValueError("Input mesh must be a single Trimesh object. Ensure the file contains a single mesh.")
    
    # Invert the face orientation
    mesh.invert()
    
    # Optional: Apply smoothing
    #try:
    #    mesh.vertices = trimesh.smoothing.filter_laplacian(mesh.vertices, iterations=2)
    #except AttributeError:
        #print("Smoothing skipped due to unsupported operation on this mesh format.")

    # Simplify the mesh if target_faces is specified
    target_faces = int(len(mesh.faces) * (reduction_factor))
    mesh = mesh.simplify_quadratic_decimation(target_faces)
    
    if not mesh.is_watertight:
        print("Mesh is not watertight. Attempting to fix...")
        bo = mesh.fill_holes()
        if not bo:
            print("Warning: The mesh is still not fully watertight after repair.")
        else:
            print("Successfully made the mesh watertight.")
    #else:
        #print("Mesh is already watertight.")
    # Save the processed mesh
    mesh.export(output_path)
    print(f"Processed mesh saved to {output_path}")

def vtpToObj(file_path):
    # Read the .vtp file
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()

    poly_data = reader.GetOutput()

    # Write the .obj file
    obj_writer = vtk.vtkOBJWriter()
    obj_writer.SetInputData(poly_data)
    obj_writer.SetFileName(file_path.split(".")[0] + ".obj")
    obj_writer.Write()

    print(f"Successfully exported {file_path} to OBJ format.")

In [3]:
'''segments = [np.array([[ 0.00148943, -0.00337053,  0.00134013],
       [-0.00166719,  0.00097624, -0.00200333],
       [ 0.00515626, -0.00231188, -0.00814563],
       [ 0.00342129, -0.00400993, -0.00736581],
       [ 0.0047332 , -0.0044332 , -0.00973243],
       [ 0.00468772, -0.00276337, -0.01069856],
       [ 0.00236762, -0.00057082, -0.0027196 ]]), np.array([[ 0.00236762, -0.00057082, -0.0027196 ],
       [ 0.00030312,  0.00209354, -0.00043037],
       [ 0.00148943, -0.00337053,  0.00134013],
       [-0.0017227 ,  0.00238752, -0.00100396],
       [-0.00166719,  0.00097624, -0.00200333],
       [-0.00032672,  0.0008851 , -0.00161559]]), np.array([[-0.00166719,  0.00097624, -0.00200333],
       [ 0.00148943, -0.00337053,  0.00134013]]), np.array([[ 0.00148943, -0.00337053,  0.00134013],
       [ 0.00236762, -0.00057082, -0.0027196 ],
       [-0.00032672,  0.0008851 , -0.00161559],
       [-0.00373699, -0.00024663, -0.00296744],
       [-0.00166719,  0.00097624, -0.00200333]])]
radii =  [np.array([[-3.65049636e-05],
       [ 9.23796168e-04],
       [ 1.72606087e-03],
       [ 1.29705772e-03],
       [ 1.45279383e-03],
       [ 1.32499193e-03],
       [ 8.97863676e-04]]), np.array([[ 8.97863676e-04],
       [ 1.11723924e-03],
       [-3.65049636e-05],
       [ 7.92136358e-04],
       [ 9.23796168e-04],
       [ 8.09013240e-04]]), np.array([[ 9.23796168e-04],
       [-3.65049636e-05]]), np.array([[-3.65049636e-05],
       [ 8.97863676e-04],
       [ 8.09013240e-04],
       [ 1.09467260e-03],
       [ 9.23796168e-04]])]
# Example usage:
segments = [
    np.array([[0.00448942, -0.00091406, 0.00230632],
              [0.00568946, -0.00027821, 0.00341608],
              [0.00073165, 0.00258207, 0.00205037],
              [0.00300354, -0.00233369, 0.000325  ]]),
    np.array([[0.00300354, -0.00233369, 0.000325  ],
              [0.00073165, 0.00258207, 0.00205037],
              [0.00304283, -0.00629337, 0.00081792]])
]
radii = [
    np.array([0.01, 0.02, 0.015, 0.012]),
    np.array([0.012, 0.015, 0.011])
]
'''
segments, radii = load_segments_and_radii_from_text("segments.txt", "radii.txt")
polydata = create_vtk_polydata(segments, radii)


    
# Optional: Write the polydata to a file (e.g., VTK format)
writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName("output.vtp")
writer.SetInputData(polydata)
writer.Write()

1

In [4]:
centerline_modeller = vmtkscripts.vmtkCenterlineModeller()
#centerline_modeller.Centerlines = read_vtk("centerline2.vtp")
centerline_modeller.dimensions = [256, 256, 256]
centerline_modeller.Centerlines = polydata
centerline_modeller.RadiusArrayName = "Radius"
centerline_modeller.Execute()
					
##marching cubes to generate mesh
mc = vmtkscripts.vmtkMarchingCubes()
mc.Image = centerline_modeller.Image
mc.Execute()

myWriter = vmtkscripts.vmtkSurfaceWriter()
myWriter.Surface = mc.Surface
#myWriter.OutputFileName = 'generadas2/'+dataset_name+ "P" +p + "eps" + eps+'/' + str(i) +'.vtp'
myWriter.OutputFileName = 'malla.vtp'
#myWriter.OutputFileName = 'vesselgpt/1.vtp'
myWriter.Execute()
			
#vtpToObj("Sinha/MallasCortadas/" + dataset_name  + "P10eps02/"+ file.split(".")[0]+'.vtp')
#process_mesh( "Sinha/MallasCortadas/" + dataset_name  + "P10eps02/"+ file.split(".")[0]+'.obj', "Sinha/MallasCortadas/" + dataset_name  + "P10eps02/watertight_meshes/X_"+ str(i) +'.obj')

Writing VTK XML surface file.


In [5]:
seg = "generados/aneurisk/segments"
rad = "generados/aneurisk/features"
segments = os.listdir(seg)
radii = os.listdir(rad)
center = "generados/aneurisk/centerlines"
mallas = "generados/aneurisk/mallasKuipersAneurisk"

for segment, radius in zip(segments, radii):
    print(segment)
    loaded_segments, loaded_radii = load_segments_and_radii_from_text(seg +"/" +segment, rad +"/" +radius)
    polydata = create_vtk_polydata(loaded_segments, loaded_radii)
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(center +"/" +segment.split(".")[0] + ".vtp")
    writer.SetInputData(polydata)
    writer.Write()
    centerline_modeller = vmtkscripts.vmtkCenterlineModeller()
    #centerline_modeller.Centerlines = read_vtk("centerline2.vtp")
    centerline_modeller.dimensions = [256, 256, 256]
    centerline_modeller.Centerlines = polydata
    centerline_modeller.RadiusArrayName = "Radius"
    centerline_modeller.Execute()
                        
    ##marching cubes to generate mesh
    mc = vmtkscripts.vmtkMarchingCubes()
    mc.Image = centerline_modeller.Image
    mc.Execute()

    myWriter = vmtkscripts.vmtkSurfaceWriter()
    myWriter.Surface = mc.Surface
    #myWriter.OutputFileName = 'generadas2/'+dataset_name+ "P" +p + "eps" + eps+'/' + str(i) +'.vtp'
    myWriter.OutputFileName = mallas +"/" +segment.split(".")[0] + ".vtp"
    #myWriter.OutputFileName = 'vesselgpt/1.vtp'
    myWriter.Execute()
                
    vtpToObj(mallas +"/" +segment.split(".")[0] + ".vtp")
    process_mesh( mallas +"/" +segment.split(".")[0] + ".obj", mallas +"/" +segment.split(".")[0] + ".obj")
        


segments_6.txt
Writing VTK XML surface file.
Successfully exported generados/aneurisk/mallasKuipersAneurisk/segments_6.vtp to OBJ format.


/tmp/ipykernel_3887795/2916588284.py:108: DeprecationWarning: `simplify_quadratic_decimation` is deprecated as it was a typo and will be removed in March 2024: replace with `simplify_quadric_decimation`
  mesh = mesh.simplify_quadratic_decimation(target_faces)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Processed mesh saved to generados/aneurisk/mallasKuipersAneurisk/segments_6.obj
segments_2.txt
Writing VTK XML surface file.
Successfully exported generados/aneurisk/mallasKuipersAneurisk/segments_2.vtp to OBJ format.
Processed mesh saved to generados/aneurisk/mallasKuipersAneurisk/segments_2.obj
segments_8.txt


/tmp/ipykernel_3887795/2916588284.py:108: DeprecationWarning: `simplify_quadratic_decimation` is deprecated as it was a typo and will be removed in March 2024: replace with `simplify_quadric_decimation`
  mesh = mesh.simplify_quadratic_decimation(target_faces)


Writing VTK XML surface file.
Successfully exported generados/aneurisk/mallasKuipersAneurisk/segments_8.vtp to OBJ format.
Processed mesh saved to generados/aneurisk/mallasKuipersAneurisk/segments_8.obj
segments_3.txt
Writing VTK XML surface file.
Successfully exported generados/aneurisk/mallasKuipersAneurisk/segments_3.vtp to OBJ format.
Mesh is not watertight. Attempting to fix...
Processed mesh saved to generados/aneurisk/mallasKuipersAneurisk/segments_3.obj
segments_5.txt
Writing VTK XML surface file.
Successfully exported generados/aneurisk/mallasKuipersAneurisk/segments_5.vtp to OBJ format.
Processed mesh saved to generados/aneurisk/mallasKuipersAneurisk/segments_5.obj
segments_4.txt
Writing VTK XML surface file.
Successfully exported generados/aneurisk/mallasKuipersAneurisk/segments_4.vtp to OBJ format.
Processed mesh saved to generados/aneurisk/mallasKuipersAneurisk/segments_4.obj
segments_0.txt
Writing VTK XML surface file.
Successfully exported generados/aneurisk/mallasKuiper